In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchsummary import summary
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(size=224, interpolation=torchvision.transforms.InterpolationMode.BICUBIC),    
        transforms.RandomPerspective(distortion_scale=0.1, p=0.5),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = ''

train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train'])
val_datasets = datasets.ImageFolder(os.path.join(data_dir, 'val'), data_transforms['val'])
# test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), data_transforms['test'])

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=32, shuffle=True, num_workers=4)
val_dataloader = torch.utils.data.DataLoader(val_datasets, batch_size=32, shuffle=True, num_workers=4)
# test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=32, shuffle=True, num_workers=4)

class_names = train_datasets.classes

In [ ]:
def evaluation(model, dataloader):
    model.eval()
    dataset_size = len(dataloader.dataset)
    running_loss = 0.0
    running_corrects = 0
    
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
                
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
    
    epoch_loss = running_loss / dataset_size
    epoch_acc = running_corrects.double() / dataset_size
    return epoch_acc, epoch_loss

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0
    val_epoch_acc = 0 
    train_dataset_size = len(train_dataloader.dataset)
    train_loss = []
    val_loss = []
    train_acc = []
    val_acc = []
   
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        
        running_loss = 0.0
        running_corrects = 0
        i = 0
            
        model.train() 
        # Iterate over data during the training stage.
        for inputs, labels in train_dataloader:
            i = i+1
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # backward + optimize  phase
                loss.backward()
                optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
        scheduler.step()
                
        train_epoch_loss = running_loss / train_dataset_size
        train_loss.append(train_epoch_loss)
        train_epoch_acc = running_corrects.double() / train_dataset_size
        train_acc.append(train_epoch_acc.item())
        print(f'Train loss: {train_epoch_loss:.4f} Training_acc: {train_epoch_acc:.4f}')
        
        val_since = time.time()
        val_epoch_acc, val_epoch_loss = evaluation(model, val_dataloader)
        val_loss.append(val_epoch_loss)
        val_acc.append(val_epoch_acc.item())
        val_time = time.time() - val_since
        print(f'Val complete in {val_time:4f} seconds.')
        print(f'Val loss: {val_epoch_loss:.4f} Val acc: {val_epoch_acc:.4f}')
        
        # deep copy the model
        if  val_epoch_acc > best_val_acc:
            best_val_acc = val_epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        print(i)

    time_elapsed = time.time() - since
    print(f'Training and validation complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.4f}s')
    print(f'Best Val Acc: {best_val_acc:4f}')
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_loss, train_acc, val_loss, val_acc

In [ ]:
def loss_plot(train_loss, val_loss, index, epoch_num, lr):
    x = np.arange(epoch_num)
    plt.plot(x, train_loss, 'r-+', label='Train loss')
    plt.plot(x, val_loss, 'b-o', label='Val loss')  
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    title = 'Train and val loss with ' + str(index) + ' layer(s) locked and learning rate set to ' + str(lr)
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
def acc_plot(train_acc, val_acc, index, epoch_num, lr):
    x = np.arange(epoch_num)
    plt.plot(x, train_acc, 'r-+', label='Train acc')
    plt.plot(x, val_acc, 'b-o', label='Val acc')  
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    title = 'Train and val acc with ' + str(index) + ' layer(s) locked and learning rate set to ' + str(lr)
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
best_test_acc_list = []
epoch_num = 30
criterion = nn.CrossEntropyLoss()

In [ ]:
learning_rate = [1e-4, 5e-5, 3e-5, 1e-5, 5e-6, 1e-6]
train_loss_list = []
val_loss_list = []
train_acc_list = []
val_acc_list = []

In [ ]:
vgg16 = models.vgg16(pretrained=True)
vgg16.classifier[3] = nn.Linear(4096, 1000)
vgg16.classifier[6] = nn.Linear(1000, 2)
nn.init.xavier_uniform_(vgg16.classifier[3].weight)
nn.init.xavier_uniform_(vgg16.classifier[6].weight)
vgg16 = vgg16.to(device)

In [ ]:
vgg16

In [ ]:
for lr in learning_rate:
    print(lr)
    
    vgg16 = models.vgg16(pretrained=True)
    vgg16.classifier[3] = nn.Linear(4096, 1000)
    vgg16.classifier[6] = nn.Linear(1000, 2)
    nn.init.xavier_uniform_(vgg16.classifier[3].weight)
    nn.init.xavier_uniform_(vgg16.classifier[6].weight)
    vgg16 = vgg16.to(device)
    
    optimizer = torch.optim.Adam([{'params': vgg16.features.parameters()}, 
        {'params': vgg16.classifier.parameters(), 'lr': lr * 10}], 
        lr=lr, weight_decay=0.001)
    # Decay LR by a factor of 0.1 every 10 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    vgg16, train_loss, train_acc, val_loss, val_acc = train_model(vgg16, criterion, optimizer, exp_lr_scheduler, epoch_num)
    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)

In [ ]:
for i in range(6):
    loss_plot(train_loss_list[i], val_loss_list[i], 0, epoch_num, learning_rate[i])

In [ ]:
for i in range(6):
    acc_plot(train_acc_list[i], val_acc_list[i], 0, epoch_num, learning_rate[i])